# 02. C - Current Earnings (현재 분기 실적 분석)

## 학습 목표
- 분기별 EPS 성장률 계산 방법
- 매출 성장률 분석
- 실적 가속화 패턴 감지
- 한국 시장에서의 분기 실적 데이터 처리

## CAN SLIM 'C' 기준
- **분기 EPS 성장률**: 전년 동기 대비 25% 이상 (권장: 50% 이상)
- **분기 매출 성장률**: 전년 동기 대비 25% 이상
- **가속하는 성장**: 최근 3분기 연속 EPS 성장률 증가

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from pykrx import stock
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# 날짜 설정
end_date = datetime.now()
start_date = end_date - timedelta(days=365*3)  # 3년 데이터 (분기 분석용)

start_str = start_date.strftime('%Y%m%d')
end_str = end_date.strftime('%Y%m%d')

print(f"분석 기간: {start_str} ~ {end_str}")

분석 기간: 20220906 ~ 20250905


## 1. 분기별 재무 데이터 수집 및 정리

In [2]:
def get_quarterly_financials(ticker, start, end):
    """
    분기별 재무 데이터 수집 및 정리
    
    Note: pykrx는 일별 데이터를 제공하므로 분기별로 집계 필요
    """
    # 기본 재무 데이터
    fund_data = stock.get_market_fundamental_by_date(start, end, ticker)
    
    # 주가 데이터 (매출 추정용)
    price_data = stock.get_market_ohlcv_by_date(start, end, ticker)
    
    # 분기별 집계
    fund_data['year'] = fund_data.index.year
    fund_data['quarter'] = fund_data.index.quarter
    fund_data['year_quarter'] = fund_data['year'].astype(str) + 'Q' + fund_data['quarter'].astype(str)
    
    # 각 분기의 마지막 값 사용
    quarterly_data = fund_data.groupby('year_quarter').last()
    
    # 시가총액으로 매출 추정 (PER과 EPS 활용)
    quarterly_data['market_cap'] = quarterly_data['EPS'] * quarterly_data['PER'] * fund_data.groupby('year_quarter')['PER'].count()
    quarterly_data['estimated_sales'] = quarterly_data['market_cap'] / quarterly_data['PER'] * 10  # 추정치
    
    return quarterly_data

# 삼성전자 분기 데이터
samsung_quarterly = get_quarterly_financials('005930', start_str, end_str)
print("삼성전자 분기별 데이터:")
print(samsung_quarterly[['EPS', 'PER', 'PBR', 'estimated_sales']].tail(8))

삼성전자 분기별 데이터:
               EPS    PER   PBR  estimated_sales
year_quarter                                    
2023Q4        8057   9.74  1.54        4834200.0
2024Q1        8057  10.23  1.62        4914770.0
2024Q2        2131  38.24  1.57        1278600.0
2024Q3        2131  28.86  1.18        1321220.0
2024Q4        2131  24.96  1.02        1299910.0
2025Q1        2131  27.12  1.11        1235980.0
2025Q2        4950  12.08  1.03        2970000.0
2025Q3        4950  14.12  1.21        2376000.0


## 2. EPS 성장률 계산

In [3]:
def calculate_eps_growth(quarterly_data):
    """
    분기별 EPS 성장률 계산
    """
    df = quarterly_data.copy()
    
    # 전년 동기 대비 성장률 (YoY)
    df['EPS_YoY'] = df['EPS'].pct_change(periods=4) * 100
    
    # 전분기 대비 성장률 (QoQ)
    df['EPS_QoQ'] = df['EPS'].pct_change(periods=1) * 100
    
    # 성장률 가속화 체크 (최근 3분기)
    df['EPS_Acceleration'] = df['EPS_YoY'].diff()
    
    # 3분기 연속 성장 가속화 체크
    df['Consecutive_Acceleration'] = (df['EPS_Acceleration'] > 0).rolling(window=3).sum() == 3
    
    return df

samsung_growth = calculate_eps_growth(samsung_quarterly)
print("EPS 성장률 분석:")
print(samsung_growth[['EPS', 'EPS_YoY', 'EPS_QoQ', 'EPS_Acceleration', 'Consecutive_Acceleration']].tail(8))

# CAN SLIM 기준 충족 확인
latest_eps_growth = samsung_growth['EPS_YoY'].iloc[-1]
print(f"\n최근 분기 EPS YoY 성장률: {latest_eps_growth:.1f}%")
print(f"CAN SLIM 기준 (25% 이상): {'✅ 충족' if latest_eps_growth >= 25 else '❌ 미충족'}")
print(f"우수 기준 (50% 이상): {'✅ 충족' if latest_eps_growth >= 50 else '❌ 미충족'}")

EPS 성장률 분석:
               EPS     EPS_YoY     EPS_QoQ  EPS_Acceleration  \
year_quarter                                                   
2023Q4        8057   39.466851    0.000000          0.000000   
2024Q1        8057   39.466851    0.000000          0.000000   
2024Q2        2131  -73.550949  -73.550949       -113.017801   
2024Q3        2131  -73.550949    0.000000          0.000000   
2024Q4        2131  -73.550949    0.000000          0.000000   
2025Q1        2131  -73.550949    0.000000          0.000000   
2025Q2        4950  132.285312  132.285312        205.836262   
2025Q3        4950  132.285312    0.000000          0.000000   

              Consecutive_Acceleration  
year_quarter                            
2023Q4                           False  
2024Q1                           False  
2024Q2                           False  
2024Q3                           False  
2024Q4                           False  
2025Q1                           False  
2025Q2             

## 3. 매출 성장률 분석

In [4]:
def calculate_sales_growth(quarterly_data):
    """
    분기별 매출 성장률 계산
    """
    df = quarterly_data.copy()
    
    # 매출 성장률 (추정치 기반)
    df['Sales_YoY'] = df['estimated_sales'].pct_change(periods=4) * 100
    df['Sales_QoQ'] = df['estimated_sales'].pct_change(periods=1) * 100
    
    # 매출과 이익의 동반 성장 체크
    df['Sales_EPS_Sync'] = ((df['Sales_YoY'] > 25) & (df['EPS_YoY'] > 25))
    
    return df

samsung_sales = calculate_sales_growth(samsung_growth)
print("매출 성장률 분석:")
print(samsung_sales[['estimated_sales', 'Sales_YoY', 'Sales_QoQ', 'Sales_EPS_Sync']].tail(8))

latest_sales_growth = samsung_sales['Sales_YoY'].iloc[-1]
print(f"\n최근 분기 매출 YoY 성장률: {latest_sales_growth:.1f}%")
print(f"CAN SLIM 기준 (25% 이상): {'✅ 충족' if latest_sales_growth >= 25 else '❌ 미충족'}")

매출 성장률 분석:
              estimated_sales   Sales_YoY   Sales_QoQ  Sales_EPS_Sync
year_quarter                                                         
2023Q4              4834200.0   34.967921   -3.225806            True
2024Q1              4914770.0   37.217386    1.666667            True
2024Q2              1278600.0  -73.984540  -73.984540           False
2024Q3              1321220.0  -73.550949    3.333333           False
2024Q4              1299910.0  -73.110132   -1.612903           False
2025Q1              1235980.0  -74.851722   -4.918033           False
2025Q2              2970000.0  132.285312  140.295150            True
2025Q3              2376000.0   79.833790  -20.000000            True

최근 분기 매출 YoY 성장률: 79.8%
CAN SLIM 기준 (25% 이상): ✅ 충족


## 4. 실적 가속화 패턴 분석

In [5]:
def analyze_earnings_acceleration(df):
    """
    실적 가속화 패턴 분석
    """
    analysis = {}
    
    # 최근 4분기 EPS 성장률
    recent_4q = df['EPS_YoY'].tail(4).values
    analysis['recent_4q_growth'] = recent_4q
    
    # 가속화 점수 (연속 개선 분기 수)
    acceleration_count = 0
    for i in range(1, len(recent_4q)):
        if recent_4q[i] > recent_4q[i-1]:
            acceleration_count += 1
    
    analysis['acceleration_score'] = acceleration_count / 3 * 100  # 최대 3회 비교
    
    # 평균 성장률 추세
    analysis['avg_growth_recent_2q'] = df['EPS_YoY'].tail(2).mean()
    analysis['avg_growth_previous_2q'] = df['EPS_YoY'].tail(4).head(2).mean()
    analysis['growth_trend'] = 'Accelerating' if analysis['avg_growth_recent_2q'] > analysis['avg_growth_previous_2q'] else 'Decelerating'
    
    return analysis

acceleration = analyze_earnings_acceleration(samsung_sales)
print("실적 가속화 분석:")
print(f"최근 4분기 EPS 성장률: {acceleration['recent_4q_growth']}")
print(f"가속화 점수: {acceleration['acceleration_score']:.0f}%")
print(f"성장 추세: {acceleration['growth_trend']}")
print(f"최근 2분기 평균: {acceleration['avg_growth_recent_2q']:.1f}%")
print(f"이전 2분기 평균: {acceleration['avg_growth_previous_2q']:.1f}%")

실적 가속화 분석:
최근 4분기 EPS 성장률: [-73.55094948 -73.55094948 132.28531206 132.28531206]
가속화 점수: 33%
성장 추세: Accelerating
최근 2분기 평균: 132.3%
이전 2분기 평균: -73.6%


## 5. 시각화

In [6]:
def visualize_earnings_growth(df, ticker_name):
    """
    실적 성장률 시각화
    """
    fig = make_subplots(
        rows=3, cols=1,
        subplot_titles=('EPS 추이', 'EPS 성장률 (YoY)', '실적 가속화'),
        vertical_spacing=0.1
    )
    
    # 1. EPS 추이
    fig.add_trace(
        go.Bar(x=df.index, y=df['EPS'], name='EPS',
               marker_color='lightblue'),
        row=1, col=1
    )
    
    # 2. EPS 성장률
    colors = ['green' if x >= 25 else 'red' for x in df['EPS_YoY'].fillna(0)]
    fig.add_trace(
        go.Bar(x=df.index, y=df['EPS_YoY'], name='EPS YoY %',
               marker_color=colors),
        row=2, col=1
    )
    
    # 기준선 추가
    fig.add_hline(y=25, line_dash="dash", line_color="orange",
                  annotation_text="CAN SLIM 기준 (25%)",
                  row=2, col=1)
    fig.add_hline(y=50, line_dash="dash", line_color="red",
                  annotation_text="우수 기준 (50%)",
                  row=2, col=1)
    
    # 3. 실적 가속화
    fig.add_trace(
        go.Scatter(x=df.index, y=df['EPS_Acceleration'],
                   mode='lines+markers', name='가속화',
                   line=dict(color='purple', width=2)),
        row=3, col=1
    )
    
    fig.add_hline(y=0, line_dash="solid", line_color="gray",
                  row=3, col=1)
    
    # 레이아웃 설정
    fig.update_layout(
        title=f'{ticker_name} - C (Current Earnings) 분석',
        height=900,
        showlegend=False
    )
    
    fig.update_xaxes(title_text="분기", row=3, col=1)
    fig.update_yaxes(title_text="EPS", row=1, col=1)
    fig.update_yaxes(title_text="성장률(%)", row=2, col=1)
    fig.update_yaxes(title_text="가속도(%p)", row=3, col=1)
    
    fig.show()

# 시각화
visualize_earnings_growth(samsung_sales.tail(12), '삼성전자')

## 6. 여러 종목 스크리닝

In [7]:
def screen_current_earnings(tickers, start, end):
    """
    여러 종목의 C 지표 스크리닝
    """
    results = []
    
    for ticker in tickers:
        try:
            name = stock.get_market_ticker_name(ticker)
            quarterly = get_quarterly_financials(ticker, start, end)
            
            if len(quarterly) < 5:  # 최소 5분기 데이터 필요
                continue
            
            growth_df = calculate_eps_growth(quarterly)
            sales_df = calculate_sales_growth(growth_df)
            acceleration = analyze_earnings_acceleration(sales_df)
            
            # 최근 데이터
            latest = sales_df.iloc[-1]
            
            results.append({
                'ticker': ticker,
                'name': name,
                'EPS_YoY': latest['EPS_YoY'],
                'Sales_YoY': latest['Sales_YoY'],
                'Acceleration_Score': acceleration['acceleration_score'],
                'Growth_Trend': acceleration['growth_trend'],
                'C_Score': calculate_c_score(latest, acceleration)
            })
            
        except Exception as e:
            print(f"Error with {ticker}: {e}")
            continue
    
    return pd.DataFrame(results)

def calculate_c_score(latest_data, acceleration):
    """
    C 지표 종합 점수 계산 (0-100)
    """
    score = 0
    
    # EPS 성장률 (40점)
    if latest_data['EPS_YoY'] >= 50:
        score += 40
    elif latest_data['EPS_YoY'] >= 25:
        score += 30
    elif latest_data['EPS_YoY'] >= 15:
        score += 20
    elif latest_data['EPS_YoY'] >= 0:
        score += 10
    
    # 매출 성장률 (30점)
    if latest_data['Sales_YoY'] >= 25:
        score += 30
    elif latest_data['Sales_YoY'] >= 15:
        score += 20
    elif latest_data['Sales_YoY'] >= 5:
        score += 10
    
    # 가속화 (30점)
    score += acceleration['acceleration_score'] * 0.3
    
    return min(score, 100)

# 테스트: 주요 종목 스크리닝
test_tickers = ['005930', '000660', '035420', '051910', '006400']  # 삼성전자, SK하이닉스, 네이버, LG화학, 삼성SDI
c_screening = screen_current_earnings(test_tickers, start_str, end_str)

print("C 지표 스크리닝 결과:")
print(c_screening.sort_values('C_Score', ascending=False))

C 지표 스크리닝 결과:
   ticker    name     EPS_YoY   Sales_YoY  Acceleration_Score  Growth_Trend  \
0  005930    삼성전자  132.285312   79.833790           33.333333  Accelerating   
1  000660  SK하이닉스         inf  597.371588           33.333333  Accelerating   
2  035420   NAVER   90.692088   47.632584           33.333333  Accelerating   
3  051910    LG화학 -100.000000   -6.451613            0.000000  Decelerating   
4  006400   삼성SDI  -70.799494  -77.393157            0.000000  Decelerating   

   C_Score  
0     80.0  
1     80.0  
2     80.0  
3      0.0  
4      0.0  


## 7. 통과 종목 필터링

In [8]:
def filter_c_criteria(screening_df, strict=True):
    """
    CAN SLIM C 기준 통과 종목 필터링
    """
    if strict:
        # 엄격한 기준: 모든 조건 충족
        filtered = screening_df[
            (screening_df['EPS_YoY'] >= 25) &
            (screening_df['Sales_YoY'] >= 25) &
            (screening_df['Growth_Trend'] == 'Accelerating')
        ]
    else:
        # 완화된 기준: 주요 조건 중 2개 이상 충족
        conditions_met = (
            (screening_df['EPS_YoY'] >= 25).astype(int) +
            (screening_df['Sales_YoY'] >= 25).astype(int) +
            (screening_df['Growth_Trend'] == 'Accelerating').astype(int)
        )
        filtered = screening_df[conditions_met >= 2]
    
    return filtered

# 엄격한 기준 적용
passed_strict = filter_c_criteria(c_screening, strict=True)
print("엄격한 기준 통과 종목:")
print(passed_strict[['ticker', 'name', 'EPS_YoY', 'Sales_YoY', 'C_Score']])

# 완화된 기준 적용
passed_relaxed = filter_c_criteria(c_screening, strict=False)
print("\n완화된 기준 통과 종목:")
print(passed_relaxed[['ticker', 'name', 'EPS_YoY', 'Sales_YoY', 'C_Score']])

엄격한 기준 통과 종목:
   ticker    name     EPS_YoY   Sales_YoY  C_Score
0  005930    삼성전자  132.285312   79.833790     80.0
1  000660  SK하이닉스         inf  597.371588     80.0
2  035420   NAVER   90.692088   47.632584     80.0

완화된 기준 통과 종목:
   ticker    name     EPS_YoY   Sales_YoY  C_Score
0  005930    삼성전자  132.285312   79.833790     80.0
1  000660  SK하이닉스         inf  597.371588     80.0
2  035420   NAVER   90.692088   47.632584     80.0


## 마무리

### 핵심 포인트
1. **분기 EPS 성장률**이 25% 이상인 종목 찾기
2. **매출 성장률**도 함께 확인하여 실적 품질 검증
3. **가속화 패턴** 확인으로 모멘텀 있는 종목 선별
4. 한국 시장 데이터의 특성을 고려한 처리

### 주의사항
- pykrx는 일별 데이터를 제공하므로 분기별 집계 필요
- 실제 분기 실적 발표일과 데이터 업데이트 시차 존재
- 업종별 특성을 고려한 기준 조정 필요

### 다음 단계
- **03_annual_earnings_analysis.ipynb**: A 지표 (연간 실적) 분석
- 장기 성장성과 안정성 평가